# Модуль 18. Практическая работа

## Цели работы

- Попрактиковаться в предобработке текстовых данных (токенизация, лемматизация, векторизация).

- Построить две модели классификации с разными способами векторизации текстов (`CountVectorizer` и `TfIdfVectorizer`).
- Сравнить качество моделей.

## Что нужно сделать

1. Загрузите датасет из файла, приложенного к заданию (`m18_jokes_dataset.csv`).

1. Закодируйте целевую переменную с помощью `LabelEncoder`.
1. Разделите выборку на тренировочную и тестовую.
1. Используя знания из модуля, подготовьте функцию `preprocess`, которая:
- удаляет стоп-слова в соответствии со списком `nltk.corpus.stopwords`;
- приводит слова к нормальной форме с помощью `pymorphy2`.
5. Создайте экзепляры векторизаторов `CountVectorizer` и `TfIdfVectorizer`. В качестве функции предобработки данных оба должны использовать функцию `preprocess`.

1. Векторизуйте признаки тренировочной и тестовой выборок. Сохраните результаты `CountVectorizer` в `X_train_count` и `X_test_count`, а результаты `TfIdfVectorizer` – в `X_train_tfidf` и `X_test_tfidf`.
1. Обучите две модели классификации, например `LinearSVC`.
1. Оцените работу обеих моделей на тестовой выборке, используя `f1_score` с методом усреднения `macro` (параметр `average='macro'`) – это арифметическое среднее f1-метрик, рассчитанных для каждого класса. 

## Что оценивается

- Все шаги выполнены правильно.

- Код решения соответствует референсному коду, который есть у куратора. 
- Обучены две модели, которые используют текстовые данные, обработанные разными векторизаторами (`CountVectorizer` и `TfIdfVectorizer`). Для обеих моделей вычислен усреднённый `f1_score` (метод усреднения — `macro`). 

## Информация о задаче

### Описание датасета

`m18_jokes_dataset.csv` – датасет, подготовленный на основе датасета, размещённого [на kaggle](https://www.kaggle.com/datasets/konstantinalbul/russian-jokes). Наш датасет содержит две колонки:

- **text** — текст анекдота (скорее всего, несмешного, но ваша задача не смеяться, а решить задачу классификации);

- **theme** — категория, к которой относится шутка. 

Датасет содержит данные для задачи многоклассовой классификации. В отличие от оригинального датасета, в нашем классов меньше, зато они идеально сбалансированы: в каждом ровно тысяча примеров. Так вам будет проще сконцентрироваться на текстовых данных.

### Порядок выполнения 

Напишите весь необходимый код в ячейках ниже.

### Рекомендации

- Обратите внимание, что модели sklearn (например, `LinearSVC`) умеют работать с разреженными матрицами (`scipy.sparse._csr.csr_matrix`), которые возвращают методы и `fit_transform` и `transform` векторизаторов `CountVectorizer` и `TfIdfVectorizer`.

- Когда подготавливаете векторизатор, вызывайте `fit_transform` на тренировочной выборке, а потом `fit` на тестовой, примерно так:
    > X_train_count = count_vectorizer.fit_transform(X_train)
    >
    > X_test_count = count_vectorizer.transform(X_test)

Затем используйте полученные результаты для обучения и контроля моделей `sklearn`.

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

#
# Ваш код здесь
#

# Решение
(для куратора)

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('data/skillbox/ml_jun18/m18_jokes_dataset.csv')

df.head()

,theme,text
0,aforizmi,"Почему удар в спину наносят те, кого,как прави..."
1,aforizmi,Музы - пожалуй самые мудрые представительницы...
2,aforizmi,"Тяжело бухать всю ночь, особенно если ночь пол..."
3,aforizmi,ПОКУШЕНИЕ. НАЛИЧНОСТЬ.\r\n\r\n\r\n
4,aforizmi,"Когда медленно танцуешь, ничего не мешает...\r..."


In [ ]:
# Датасет идеально сбалансирован
df.groupby('theme').size()

theme
aforizmi                 1000
meditsinskie             1000
narodnie                 1000
poshlie-i-intimnie       1000
pro-alkogolikov          1000
pro-armiu                1000
pro-detey                1000
pro-evreev               1000
pro-militsiyu            1000
pro-mugchin              1000
pro-novih-russkih        1000
pro-semyu                1000
pro-studentov            1000
pro-vovochku             1000
raznie                   1000
shkolnie-i-pro-shkolu    1000
tsitati                  1000
dtype: int64

In [ ]:
# Кодируем категориальную целевую переменную (метку класса),
# используя LabelEncoder

label_encoder = LabelEncoder()
df['theme'] = label_encoder.fit_transform(df['theme'])
df.sample(5)

,theme,text
7414,7,Узкая Одесская улочка. Из окна на втором этаже...
10950,10,"Едет крутой джип, весь грязный. Сзади написа..."
15859,15,Марья Ивановна спрашивает Вовочку:\r\n- Почему...
428,0,"На самом деле у меня гора мышц, вот только под..."
2483,2,"- Абраша, ты как, устроился на работу?\r\n- Да..."


In [ ]:
# Разделяем выборку на тренировочную и тестовую

X_train, X_test, y_train, y_test = train_test_split(
  df['text'], df['theme'],
  test_size=0.2
)

In [ ]:
# Создаём функцию препроцессинга, которую будем использовать в векторизаторах

tokenizer = RegexpTokenizer('\w+')
russian_stopwords = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

def preprocess(text):
    stemmed_words = []
    for word in tokenizer.tokenize(text):
        word = word.lower()
        if word not in russian_stopwords:
            stemmed_words.append(morph.parse(word)[0].normal_form)
    return ' '.join(stemmed_words)

In [ ]:
# Создаём векторизаторы с одинаковым препроцессингом

count_vectorizer = CountVectorizer(
    preprocessor=preprocess
)

tfidf_vectorizer = TfidfVectorizer(
    preprocessor=preprocess
)

In [ ]:
%%time

# Векторизуем текст, используя CountVectorizer
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

CPU times: user 43.4 s, sys: 216 ms, total: 43.6 s
Wall time: 43.9 s


In [ ]:
%%time

# Векторизуем текст, используя TfidfVectorizer
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

CPU times: user 43.2 s, sys: 176 ms, total: 43.4 s
Wall time: 43.6 s


In [ ]:
svc_count = LinearSVC(max_iter=10000)

svc_count.fit(X_train_count, y_train)

y_pred_count = svc_count.predict(X_test_count)

print(classification_report(y_test, y_pred_count))

              precision    recall  f1-score   support

           0       0.39      0.65      0.49       198
           1       0.52      0.51      0.51       209
           2       0.48      0.39      0.43       201
           3       0.39      0.37      0.38       215
           4       0.89      0.84      0.86       196
           5       0.65      0.68      0.66       197
           6       0.95      0.91      0.93       198
           7       0.78      0.81      0.80       206
           8       0.84      0.64      0.73       182
           9       0.32      0.35      0.33       211
          10       0.75      0.80      0.78       163
          11       0.36      0.46      0.40       192
          12       0.75      0.74      0.74       223
          13       0.87      0.85      0.86       192
          14       0.12      0.13      0.13       183
          15       0.97      0.87      0.91       226
          16       0.68      0.34      0.45       208

    accuracy              

In [ ]:
f1_score(y_test, y_pred_count, average='macro')

0.6117208815090223

In [ ]:
%%time

svc_tfidf = LinearSVC(max_iter=10000)

svc_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = svc_tfidf.predict(X_test_tfidf)

print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.39      0.42      0.40       198
           1       0.58      0.60      0.59       209
           2       0.51      0.50      0.51       201
           3       0.44      0.50      0.47       215
           4       0.86      0.86      0.86       196
           5       0.69      0.74      0.72       197
           6       0.91      0.92      0.91       198
           7       0.80      0.85      0.82       206
           8       0.82      0.69      0.75       182
           9       0.36      0.34      0.35       211
          10       0.74      0.87      0.80       163
          11       0.40      0.49      0.44       192
          12       0.78      0.74      0.76       223
          13       0.85      0.88      0.86       192
          14       0.14      0.11      0.12       183
          15       0.95      0.88      0.92       226
          16       0.63      0.49      0.55       208

    accuracy              

In [ ]:
f1_score(y_test, y_pred_tfidf, average='macro')

0.6373363554759008